<a href="https://colab.research.google.com/github/sincanmaulanaa/dicoding-machine-learning-development-exercises/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-play-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 906.9 kB/s eta 0:00:00


In [2]:
# Mengimpor pustaka google_play_scraper untuk mengakses ulasan dan informasi aplikasi dari Google Play Store.
from google_play_scraper import app, reviews, Sort, reviews_all
import pandas as pd # Pandas untuk manipulasi dan analisis data
pd.options.mode.chained_assignment = None # Menonaktifkan peringatan chaining

import numpy as np # NumPy untuk komputasi numerik
seed = 0
np.random.seed(seed) # Mengatur seed untuk reproduktibilitas

import matplotlib.pyplot as plt #Matplotlib untuk visualisasi data
import seaborn as sns # Seaborn untuk visualisasi data statistik, mengatur gaya visualisasi

import datetime as dt # Manipulasi waktu dan tanggal
import re # Modul untuk bekerja dengan ekspresi reguler
import string # Berisi konstanta string, seperti tanda baca

from nltk.tokenize import word_tokenize # Tokenisasi teks
from nltk.corpus import stopwords # Daftar kata-kata berhenti dalam teks

In [3]:
!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory # Stemming (penghilangan imbuhan kata) dalam bahasa Indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory # Menghapus kata-kata berhenti dalam bahasa Indonesia

from wordcloud import WordCloud # Membuat visualisasi berbentuk awan kata (word cloud) dari teks

import nltk # Import pustaka NLTK (Natural Language Toolkit)
nltk.download('punkt') # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords') # Mengunduh dataset yang berisi daftar kata-kata berhenti (stopwords) dalam berbagai bahasa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 1.5 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
# Mengambil semua ulasan dari aplikasi dengan ID 'com.byu.id' di Google Play Store.
# Proses scraping mungkin memerlukan beberapa saat tergantung pada jumlah ulasan yang ada.
scrapereview = reviews_all(
    'com.byu.id', # ID Aplikasi
    lang='id', # bahasa ulasan, default 'en'
    country='id', # negara, default 'us'
    sort=Sort.MOST_RELEVANT, # urutan ulasan, default Sort.MOST_RELEVANT
    count=1000 # jumlah maksimum ulasan yang ingin diambil
)

In [5]:
# Menyimpan ulasan dalam file CSV
import csv

with open('ulasan_aplikasi.csv', mode='w', newline='', encoding='utf-8') as file:
  writer = csv.writer(file)
  writer.writerow(['Review']) # menulis header kolom
  for review in scrapereview:
    writer.writerow([review['content']]) # menulis konten ulasan ke dalam file CSV

In [9]:
# meload dataset
app_reviews_df = pd.DataFrame(scrapereview)
app_reviews_df.shape
app_reviews_df.head()
app_reviews_df.to_csv('ulasan_aplikasi.csv', index=False)

# membuat dataframe dari hasil scrapereview
app_reviews_df = pd.DataFrame(scrapereview)

# menghitung jumlah baris dan kolom dalam dataframe
jumlah_ulasan, jumlah_kolom = app_reviews_df.shape
print('JUMLAH ULASAN: ', jumlah_ulasan)
print('JUMLAH KOLOM: ', jumlah_kolom)

JUMLAH ULASAN:  94500
JUMLAH KOLOM:  11


In [8]:
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,5731e6bb-9512-4dd2-9103-79e99be47097,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"TOLONG DI PERBAIKI LAGI APLIKASI NYA, jaringan...",1,105,1.56.0,2024-08-30 15:32:25,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2024-08-31 04:13:12,1.56.0
1,7e64ebf7-4bd7-4155-a3af-d1a377193556,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Dari awal pake by.u jaringannya bagus bgt, eh ...",1,327,1.56.0,2024-08-16 15:22:21,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2024-08-16 16:46:35,1.56.0
2,9d69b674-a5fd-47a3-bc3a-933f92cb814e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"jujur aplikasinya bagus, kartu bagus, cuman ad...",1,117,1.56.0,2024-08-20 16:40:41,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2024-08-20 16:55:52,1.56.0
3,7b82033d-0606-4753-b1ec-b19df7f810b6,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"G ada perubahan, sinyal lag banget sampe 4 jam...",1,4,1.56.0,2024-09-04 13:17:00,"Hai Kak Reza, maaf udah bikin ga nyaman ya :( ...",2024-09-04 19:38:52,1.56.0
4,a468cdd6-203b-4360-97a7-31ea50fb2ced,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Aplikasi Error, saat mau mengirim tangkapan la...",2,40,1.56.0,2024-08-17 14:58:40,"Hi, Kak mohon maaf banget udah bikin ga nyaman...",2024-08-17 19:47:43,1.56.0


In [10]:
app_reviews_df.info()

# terdapat kolom dengan nilai null, yaitu kolom
# reviewCreatedVersion, replyContent, repliedAt, appVersion

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94500 entries, 0 to 94499
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              94500 non-null  object        
 1   userName              94500 non-null  object        
 2   userImage             94500 non-null  object        
 3   content               94500 non-null  object        
 4   score                 94500 non-null  int64         
 5   thumbsUpCount         94500 non-null  int64         
 6   reviewCreatedVersion  77687 non-null  object        
 7   at                    94500 non-null  datetime64[ns]
 8   replyContent          86990 non-null  object        
 9   repliedAt             86990 non-null  datetime64[ns]
 10  appVersion            77687 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 7.9+ MB


In [11]:
# Membuat DataFrame baru (clean_df) dengan menghapus baris
# yang memiliki nilai yang hilang (NaN) dari app_reviews_df
clean_df = app_reviews_df.dropna()

In [12]:
# menghapus baris duplikat dari dataframe clean_df
clean_df = clean_df.drop_duplicates()

In [14]:
# Menghitung jumlah baris dan kolom dalam DataFrame clean_df setelah menghapus duplikat
jumlah_ulasan_setelah_hapus_duplikat, jumlah_kolom_setelah_hapus_duplikat = clean_df.shape
print("JUMLAH ULASAN: ", jumlah_ulasan_setelah_hapus_duplikat)
print("JUMLAH KOLOM: ", jumlah_kolom_setelah_hapus_duplikat)

JUMLAH ULASAN:  71180
JUMLAH KOLOM:  11


In [15]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71180 entries, 0 to 94499
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              71180 non-null  object        
 1   userName              71180 non-null  object        
 2   userImage             71180 non-null  object        
 3   content               71180 non-null  object        
 4   score                 71180 non-null  int64         
 5   thumbsUpCount         71180 non-null  int64         
 6   reviewCreatedVersion  71180 non-null  object        
 7   at                    71180 non-null  datetime64[ns]
 8   replyContent          71180 non-null  object        
 9   repliedAt             71180 non-null  datetime64[ns]
 10  appVersion            71180 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 6.5+ MB


In [19]:
# pre-processing text
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer import StemmerFactory

# Membersihkan teks dengan menghapus mention, hashtag, RT (retweet), tautan (link),
# angka, dan tanda baca. Selain itu, karakter newline diganti dengan spasi dan spasi
# ekstra pada awal dan akhir teks dihapus.

def cleaningText(text):
  text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
  text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
  text = re.sub(r'RT[\s]', '', text) # menghapus RT
  text = re.sub(r"http\S+", '', text) # menghapus link
  text = re.sub(r'[0-9]+', '', text) # menghapus angka
  text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka

  text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
  text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
  text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
  return text

def casefoldingText(text):
  text = text.lower() # mengubah teks menjadi huruf kecil
  return text

def tokenizingText(text):
  text = word_tokenize(text) # memecah atau membagi string teks menjadi token
  return text

def filteringText(text): # menghapus stopwords dalam teks
  listStopWords = set(stopwords.words('indonesian'))
  listStopWords1 = set(stopwords.words('english'))

  listStopWords.update(listStopWords1)
  listStopWords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])

  filtered = []
  for txt in text:
    if txt not in listStopWords:
      filtered.append(txt)

  text = filtered
  return text

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
  # membuat object stemmer
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()

  # memecah teks menjadi daftar kata
  words = text.split()

  # menerapkan stemming pada setiap kata dalam daftar
  stemmed_words = [stemmer.stem(word) for word in words]

  # menggabungkan kata-kata yang telah distem
  stemmed_text = ' '.join(stemmed_words)
  return stemmed_text

def toSentence(list_words):
  sentence = ' '.join(word for word in list_words)
  return sentence


In [20]:
# menghapus kumpulan slang words atau kata-kata informal
slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal"}

def fix_slangwords(text):
  words = text.split()
  fixed_words = []

  for word in words:
    if word.lower() in slangwords:
      fixed_words.append(slangwords[word.lower()])
    else:
      fixed_words.append(word)

  fixed_text = ' '.join(fixed_words)
  return fixed_text

In [24]:
# mengimplementasikan semua step pre-processing text

# Membersihkan teks dan menyimpannya di kolom 'text_clean'
clean_df['text_clean'] = clean_df['content'].apply(cleaningText)

# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)

# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)

# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)

# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)

# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence)

In [25]:
clean_df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir
0,5731e6bb-9512-4dd2-9103-79e99be47097,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"TOLONG DI PERBAIKI LAGI APLIKASI NYA, jaringan...",1,105,1.56.0,2024-08-30 15:32:25,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2024-08-31 04:13:12,1.56.0,TOLONG DI PERBAIKI LAGI APLIKASI NYA jaringan ...,tolong di perbaiki lagi aplikasi nya jaringan ...,tolong di perbaiki lagi aplikasi nya jaringan ...,"[tolong, di, perbaiki, lagi, aplikasi, nya, ja...","[tolong, perbaiki, aplikasi, jaringan, bagus, ...",tolong perbaiki aplikasi jaringan bagus lancar...
1,7e64ebf7-4bd7-4155-a3af-d1a377193556,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Dari awal pake by.u jaringannya bagus bgt, eh ...",1,327,1.56.0,2024-08-16 15:22:21,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2024-08-16 16:46:35,1.56.0,Dari awal pake byu jaringannya bagus bgt eh bb...,dari awal pake byu jaringannya bagus bgt eh bb...,dari awal pake byu jaringannya bagus banget eh...,"[dari, awal, pake, byu, jaringannya, bagus, ba...","[pake, byu, jaringannya, bagus, banget, eh, bb...",pake byu jaringannya bagus banget eh bbrp kede...
2,9d69b674-a5fd-47a3-bc3a-933f92cb814e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"jujur aplikasinya bagus, kartu bagus, cuman ad...",1,117,1.56.0,2024-08-20 16:40:41,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2024-08-20 16:55:52,1.56.0,jujur aplikasinya bagus kartu bagus cuman ada ...,jujur aplikasinya bagus kartu bagus cuman ada ...,jujur aplikasinya bagus kartu bagus cuman ada ...,"[jujur, aplikasinya, bagus, kartu, bagus, cuma...","[jujur, aplikasinya, bagus, kartu, bagus, cuma...",jujur aplikasinya bagus kartu bagus cuman kelu...
3,7b82033d-0606-4753-b1ec-b19df7f810b6,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"G ada perubahan, sinyal lag banget sampe 4 jam...",1,4,1.56.0,2024-09-04 13:17:00,"Hai Kak Reza, maaf udah bikin ga nyaman ya :( ...",2024-09-04 19:38:52,1.56.0,G ada perubahan sinyal lag banget sampe jam t...,g ada perubahan sinyal lag banget sampe jam t...,g ada perubahan sinyal lag banget sampe jam ti...,"[g, ada, perubahan, sinyal, lag, banget, sampe...","[g, perubahan, sinyal, lag, banget, sampe, jam...",g perubahan sinyal lag banget sampe jam tibati...
4,a468cdd6-203b-4360-97a7-31ea50fb2ced,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Aplikasi Error, saat mau mengirim tangkapan la...",2,40,1.56.0,2024-08-17 14:58:40,"Hi, Kak mohon maaf banget udah bikin ga nyaman...",2024-08-17 19:47:43,1.56.0,Aplikasi Error saat mau mengirim tangkapan lay...,aplikasi error saat mau mengirim tangkapan lay...,aplikasi error saat mau mengirim tangkapan lay...,"[aplikasi, error, saat, mau, mengirim, tangkap...","[aplikasi, error, mengirim, tangkapan, layar, ...",aplikasi error mengirim tangkapan layar live n...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94493,c9402947-585f-4050-8f59-a9dcd674feb1,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Untuk sekarang aku baru memakai kartu by.u, ta...",5,0,1.55.0,2024-08-03 14:15:12,"Hai Kak, makasih banyak buat rating nya. Kalau...",2024-08-03 18:26:47,1.55.0,Untuk sekarang aku baru memakai kartu byu tapi...,untuk sekarang aku baru memakai kartu byu tapi...,untuk sekarang aku baru memakai kartu byu tapi...,"[untuk, sekarang, aku, baru, memakai, kartu, b...","[memakai, kartu, byu, kendala, sinyal, apalah,...",memakai kartu byu kendala sinyal apalah oke
94495,eaf43cd3-e948-4851-9aee-2c2e996bbb01,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Ingin menambahkan saran. Tolong dibuatkan mult...,5,58,1.55.0,2024-06-21 23:31:57,"Hai Kak, makasih banyak buat feedback dan sara...",2024-06-22 03:59:10,1.55.0,Ingin menambahkan sar

In [26]:
# PELABELAN
import csv
import requests
from io import StringIO

# membaca data kamus kata-kata positif dari github
lexicon_positive = dict()

# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
if response.status_code == 200:
  # jika permintaan berhasil
  # membaca teks response sebagai file csv menggunakan pembaca csv dengan pemisah koma
  reader = csv.reader(StringIO(response.text), delimiter=',')

  for row in reader:
    # mengulangi setiap baris dalam file csv
    lexicon_positive[row[0]] = int(row[1])

    # menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
  print('Failed to fetch positive lexicon data')




# membaca data kamus kata-kata negatif dari github
lexicon_negative = dict()

# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [27]:
# Fungsi untuk menentukan polaritas sentimen dari tweet

def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:

    score = 0
    # Inisialisasi skor sentimen ke 0

    for word in text:
        # Mengulangi setiap kata dalam teks

        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen

    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)

        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen

    polarity=''
    # Inisialisasi variabel polaritas

    if (score >= 0):
        polarity = 'positive'
        # Jika skor sentimen lebih besar atau sama dengan 0, maka polaritas adalah positif
    elif (score < 0):
        polarity = 'negative'
        # Jika skor sentimen kurang dari 0, maka polaritas adalah negatif

    # else:
    #     polarity = 'neutral'
    # Ini adalah bagian yang bisa digunakan untuk menentukan polaritas netral jika diperlukan

    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

In [28]:
results = clean_df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_df['polarity_score'] = results[0]
clean_df['polarity'] = results[1]
print(clean_df['polarity'].value_counts())

polarity
negative    37418
positive    33762
Name: count, dtype: int64


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Pisahkan data menjadi fitur (tweet) dan label (sentimen)
X = clean_df['text_akhir']
y = clean_df['polarity']

# Ekstraksi fitur dengan TF-IDF
tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8 )
X_tfidf = tfidf.fit_transform(X)

# Konversi hasil ekstraksi fitur menjadi dataframe
features_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())

# Menampilkan hasil ekstraksi fitur
features_df


,aja,aktivasi,aman,aneh,apk,aplikasi,aplikasinya,app,bagus,banget,...,turun,udah,udh,ulang,unlimited,update,via,wifi,yg,youtube
0,0.000000,0.0,0.0,0.0,0.0,0.522129,0.000000,0.000000,0.133129,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.229133,0.0,0.0,0.0,0.0,0.000000,0.000000,0.192188,0.228333,0.221651,...,0.0,0.000000,0.157917,0.0,0.0,0.0,0.0,0.0,0.119645,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.139815,0.187377,0.000000,0.285193,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.261237,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.252707,...,0.0,0.137603,0.000000,0.0,0.0,0.0,0.0,0.0,0.136409,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.122258,0.327692,0.000000,0.000000,0.121041,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71175,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
71176,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.208660,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
71177,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
71178,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [31]:
# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [35]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

# Membuat objek model Naive Bayes (Bernoulli Naive Bayes)
naive_bayes = BernoulliNB()

# Melatih model Naive Bayes pada data pelatihan
naive_bayes.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_nb = naive_bayes.predict(X_train.toarray())
y_pred_test_nb = naive_bayes.predict(X_test.toarray())

# Evaluasi akurasi model Naive Bayes
accuracy_train_nb = accuracy_score(y_pred_train_nb, y_train)
accuracy_test_nb = accuracy_score(y_pred_test_nb, y_test)

# Menampilkan akurasi
print('Naive Bayes - accuracy_train:', accuracy_train_nb)
print('Naive Bayes - accuracy_test:', accuracy_test_nb)

Naive Bayes - accuracy_train: 0.7886344478786176
Naive Bayes - accuracy_test: 0.7897583590896319


In [36]:
from sklearn.ensemble import RandomForestClassifier

# Membuat objek model Random Forest
random_forest = RandomForestClassifier()

# Melatih model Random Forest pada data pelatihan
random_forest.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_rf = random_forest.predict(X_train.toarray())
y_pred_test_rf = random_forest.predict(X_test.toarray())

# Evaluasi akurasi model Random Forest
accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)

# Menampilkan akurasi
print('Random Forest - accuracy_train:', accuracy_train_rf)
print('Random Forest - accuracy_test:', accuracy_test_rf)

Random Forest - accuracy_train: 0.969110002809778
Random Forest - accuracy_test: 0.8574740095532453


In [37]:
from sklearn.linear_model import LogisticRegression

# Membuat objek model Logistic Regression
logistic_regression = LogisticRegression()

# Melatih model Logistic Regression pada data pelatihan
logistic_regression.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_lr = logistic_regression.predict(X_train.toarray())
y_pred_test_lr = logistic_regression.predict(X_test.toarray())

# Evaluasi akurasi model Logistic Regression pada data pelatihan
accuracy_train_lr = accuracy_score(y_pred_train_lr, y_train)

# Evaluasi akurasi model Logistic Regression pada data uji
accuracy_test_lr = accuracy_score(y_pred_test_lr, y_test)

# Menampilkan akurasi
print('Logistic Regression - accuracy_train:', accuracy_train_lr)
print('Logistic Regression - accuracy_test:', accuracy_test_lr)

Logistic Regression - accuracy_train: 0.8774058724360776
Logistic Regression - accuracy_test: 0.8756673222815398


In [38]:
from sklearn.tree import DecisionTreeClassifier

# Membuat objek model Decision Tree
decision_tree = DecisionTreeClassifier()

# Melatih model Decision Tree pada data pelatihan
decision_tree.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_dt = decision_tree.predict(X_train.toarray())
y_pred_test_dt = decision_tree.predict(X_test.toarray())

# Evaluasi akurasi model Decision Tree
accuracy_train_dt = accuracy_score(y_pred_train_dt, y_train)
accuracy_test_dt = accuracy_score(y_pred_test_dt, y_test)

# Menampilkan akurasi
print('Decision Tree - accuracy_train:', accuracy_train_dt)
print('Decision Tree - accuracy_test:', accuracy_test_dt)

Decision Tree - accuracy_train: 0.969110002809778
Decision Tree - accuracy_test: 0.8070384939589772
